<a href="https://colab.research.google.com/github/carmen-chan/A2/blob/master/Testing_area_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##Importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Import dataset from Github repository 
url = 'https://raw.githubusercontent.com/carmen-chan/A2/master/WA_Fn-UseC_-Telco-Customer-Churn.csv'
telco = pd.read_csv(url, error_bad_lines=False)

# Check column names 
list(telco.head())

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [0]:
## Examining the first five rows of data
print(telco.head())

## For numeric attributes find the general mathematic summarising data
telco.describe()



   customerID  gender  SeniorCitizen  ... MonthlyCharges TotalCharges  Churn
0  7590-VHVEG  Female              0  ...          29.85        29.85     No
1  5575-GNVDE    Male              0  ...          56.95       1889.5     No
2  3668-QPYBK    Male              0  ...          53.85       108.15    Yes
3  7795-CFOCW    Male              0  ...          42.30      1840.75     No
4  9237-HQITU  Female              0  ...          70.70       151.65    Yes

[5 rows x 21 columns]


,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [0]:
## Renaming Columns

telco.columns = ['CustomerID', 'Gender', 'SeniorCitizen', 'Partner','Dependents', 'TenureMonths',  'PhoneService',  'MultipleLines',  'InternetService',  'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'ContractTerm', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'MonthlyChurn']

In [0]:
telco.dtypes

CustomerID           object
Gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
TenureMonths          int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
ContractTerm         object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
MonthlyChurn         object
dtype: object

Data cleaning

In [0]:
telco = telco.apply(LabelEncoder().fit_transform)

In [0]:
telco.describe()


,CustomerID,Gender,SeniorCitizen,Partner,Dependents,TenureMonths,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,ContractTerm,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,MonthlyChurn
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,3521.000000,0.504756,0.162147,0.483033,0.299588,32.371149,0.903166,0.940508,0.872923,0.790004,0.906432,0.904444,0.797104,0.985376,0.992475,0.690473,0.592219,1.574329,683.524776,3257.794122,0.265370
std,2033.283305,0.500013,0.368612,0.499748,0.458110,24.559481,0.295752,0.948554,0.737796,0.859848,0.880162,0.879949,0.861551,0.885002,0.885091,0.833755,0.491457,1.068104,478.511521,1888.693496,0.441561
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1760.500000,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,199.000000,1609.000000,0.000000
50%,3521.000000,1.000000,0.000000,0.000000,0.000000,29.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,2.000000,722.000000,3249.000000,0.000000
75%,5281.500000,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,1090.000000,4901.500000,1.000000
max,7042.000000,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,3.000000,1584.000000,6530.000000,1.000000


# Import data packages for classifiers
# Create training and test set

In [0]:
## Import decision tree classifier package and metrics to evaluate on.

from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [0]:
predictor_cols = ['Gender', 'SeniorCitizen', 'Partner','Dependents', 'TenureMonths',  'PhoneService',  'MultipleLines',  'InternetService',  'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'ContractTerm', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']
X = telco[predictor_cols]
y = telco.MonthlyChurn

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)



# Decision tree

In [0]:
dtC = DecisionTreeClassifier(criterion="entropy", max_depth=6)

dtC = dtC.fit(X_train,y_train)

y_pred_dtC = dtC.predict(X_test)

print("Accuracy of decision tree classifier:",metrics.accuracy_score(y_test, y_pred_dtC))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_dtC))
print(classification_report(y_test,y_pred_dtC))
print(confusion_matrix(y_test,y_pred_dtC))

Accuracy of decision tree classifier: 0.7963094393186657
Mean Absolute Error: 0.2036905606813343
              precision    recall  f1-score   support

           0       0.86      0.87      0.87      1061
           1       0.59      0.57      0.58       348

    accuracy                           0.80      1409
   macro avg       0.73      0.72      0.72      1409
weighted avg       0.79      0.80      0.80      1409

[[922 139]
 [148 200]]


In [0]:
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

dot_data = StringIO()
export_graphviz(dtC, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = predictor_cols,class_names=['churned','retained'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('telcodtC.png')
Image(graph.create_png())

# Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier(n_estimators=100, random_state=1)
randomforest.fit(X_train, y_train)
y_pred_randomforest = randomforest.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred_randomforest))
print(classification_report(y_test,y_pred_randomforest))
print("Accuracy of random forest classifier:", accuracy_score(y_test, y_pred_randomforest))

[[951 110]
 [158 190]]
              precision    recall  f1-score   support

           0       0.86      0.90      0.88      1061
           1       0.63      0.55      0.59       348

    accuracy                           0.81      1409
   macro avg       0.75      0.72      0.73      1409
weighted avg       0.80      0.81      0.80      1409

Accuracy of random forest classifier: 0.8097941802696949


# Linear SVM

In [0]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

y_pred_SVM = svclassifier.predict(X_test)

print(confusion_matrix(y_test,y_pred_SVM))
print(classification_report(y_test,y_pred_SVM))
print("Accuracy of SVM classifier:", accuracy_score(y_test, y_pred_SVM))

[[941 120]
 [154 194]]
              precision    recall  f1-score   support

           0       0.86      0.89      0.87      1061
           1       0.62      0.56      0.59       348

    accuracy                           0.81      1409
   macro avg       0.74      0.72      0.73      1409
weighted avg       0.80      0.81      0.80      1409

Accuracy of SVM classifier: 0.8055358410220014


Kernel SVM

In [0]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='poly', degree=8)
svclassifier.fit(X_train, y_train)

y_pred_SVC = svclassifier.predict(X_test)

print(confusion_matrix(y_test,y_pred_SVC))
print(classification_report(y_test,y_pred_SVC))
print("Accuracy of SVC classifier:", accuracy_score(y_test, y_pred_SVC))

[[   0 1061]
 [   0  348]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1061
           1       0.25      1.00      0.40       348

    accuracy                           0.25      1409
   macro avg       0.12      0.50      0.20      1409
weighted avg       0.06      0.25      0.10      1409

Accuracy of SVC classifier: 0.24698367636621718


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
